# Preprocess

テキストや音声、画像をモデルに渡して推論・訓練を行うためには、事前にそれらをモデルが期待する形式に変換しておく必要があります。
本チュートリアルでは、`Transformers` ライブラリが提供する、データの事前処理の手法について学びます。

本チュートリアルは、[Hugging Face Transformers チュートリアル](https://huggingface.co/docs/transformers/v4.57.1/ja/preprocessing) を元に、一部加筆・修正して作成しています。

## Dependencies

このチュートリアルコードをすべて実行するためには、明示的に `import` するライブラリの他に、以下のソフトウェアが必要です。

- [`tesseract`](https://github.com/tesseract-ocr/tesseract) (および、その Python ラッパー: `pytesseract`): 動画処理
- `torch` ライブラリ or `tensorflow` ライブラリ: バックエンド
    - 本チュートリアルでは `torch` を用いるコードしか紹介しません

In [ ]:
# run this cell if you are working in google colab

%pip install pytesseract torchcodec

In [ ]:
from datasets import load_dataset, Audio
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import (
    ColorJitter,
    Compose,
    RandomResizedCrop,
)
from transformers import (
    AutoConfig,
    AutoFeatureExtractor,
    AutoImageProcessor,
    AutoProcessor,
    AutoTokenizer,
)

## Natural Language Processing

自然言語処理のタスクにおいて、テキストの事前処理に使用する主なアーキテクチャは**トーカナイザ**です。
トーカナイザは、テキストを一定のルールのもとで**トークン**に分割します。
トークンは単語や文字、あるいは単語を構成する部分文字列で構成されます。
個々のトークンに識別番号を振ることでテキストが数列に変換され、これにより、機械学習モデルが文字列を数理的に取り扱えるようになります。

ここでも、`from_pretrained()` メソッドを使用します。

In [ ]:
# model: google-bert/bert-base-cased (110M params)
# ref: https://huggingface.co/google-bert/bert-base-cased

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input)

ここで、トーカナイザの出力について補足しておきます。

- `input_ids`: 文中の各トークンに対応するインデックス
- `token_type_ids`: 複数の文が入力された場合に、それらを区別するために付与される id の列
- `attention_mask`: attention アーキテクチャがトークンを受け取る必要があるかを示す bool の列

`input_ids` をデコードすることで元の入力が得られます。
ここでわかるように、トーカナイザは文章に自動的に特別なトークン (`CLS`, `SEP`) を付与します。

In [ ]:
tokenizer.decode(encoded_input["input_ids"])

複数の文章の前処理を行うこともできます。

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs1 = tokenizer(batch_sentences)
print(encoded_inputs1)

### Padding

テキストは常に同じ長さ (同じトークン数) とは限りませんが、推論モデルはある特定の長さの入力しか受け付けることができません。
そこで、トーカナイザはテキストをトークン化しつつ、その長さを揃えることが期待されます。

このための戦略の1つがパディングです。
`padding=True` を指定することで、入力バッチ中の最長のテキストに合わせて、短いテキストに**パディングトークン**が追加されます。

In [ ]:
encoded_inputs2 = tokenizer(batch_sentences, padding=True)
print(encoded_inputs2)

### Truncation

入力テキストの長さが、モデルが期待する入力次元を超えてしまう場合があります。
`truncation=True` を指定することで、モデルが受け入れる最大の長さにトークン列を切り詰めます。

In [ ]:
encoded_inputs3 = tokenizer(batch_sentences, padding=True, truncation=True)
print(encoded_inputs3)

ちなみに、今回利用している "google-bert/bert-base-cased" モデルが浮き入れる最大トークン数は 512 であるため、`batch_sentences` に含まれている入力テキスト程度のトークン数では Truncation は有効に効いてきません。

In [ ]:
bert_config = AutoConfig.from_pretrained("google-bert/bert-base-cased")
print(bert_config.max_position_embeddings)

### Build tensors

`return_tensors="pt"` (`"tf"`) を指定することで、出力を `PyTorch` (`TensorFlow`) のテンソル形式に変換します。

In [ ]:
encoded_inputs4 = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_inputs4)

## Audio

音声処理タスクにおいて、音声データの事前処理に使用する主なアーキテクチャは**特徴抽出器**です。特徴抽出器は生の音声データから特徴を抽出し、それらをテンソルに変換します。

まず、入力データセットをロードします。

In [ ]:
dataset_audio = load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset_audio.features

`dataset_audio.features` によると、サンプリングレートは 8 kHz であるようです。
今回は Wav2Vec2 モデルへの入力を想定しますが、このモデルはサンプリングレート 16 kHz のデータで事前学習されているので、`dataset_audio` を 16 kHz でリサンプルしましょう。

In [ ]:
dataset_audio2 = dataset_audio.cast_column("audio", Audio(sampling_rate=16000))
dataset_audio2.features

次に、特徴抽出器を用いて入力データを正規化します。

In [ ]:
# model: facebook/wav2vec2-base (95M params)
# ref: https://huggingface.co/facebook/wav2vec2-base

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

In [ ]:
audio_input1 = [dataset_audio2[0]["audio"]["array"]]
print(feature_extractor(audio_input1))
audio_input2 = [dataset_audio2[1]["audio"]["array"]]
print(feature_extractor(audio_input2))

テキストデータと同様に、データセットに含まれる音声データの長さがすべて等しいとは限らず (`shape` メンバを参照) 、また、モデルが期待する入力の長さには限りがあります。
そこで、パディングとトランケーションを行います。
特徴抽出器では、最大サンプル長を制御するために `max_length=<number>` を指定します。

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        max_length=100000,
        truncation=True,
    )
    return inputs

In [ ]:
processed_dataset_audio = preprocess_function(dataset_audio2[:5])
print(processed_dataset_audio["input_values"][0].shape)
print(processed_dataset_audio["input_values"][1].shape)

## Computer Vision

画像処理タスクにおいて、画像データの事前処理に使用する主なアーキテクチャは**画像プロセッサ**です。
画像プロセッサは元の画像データに対してリサイズ・正規化・チャネル
補正などの処理を行い、それらをテンソルに変換します。

例によって、データセットと画像プロセッサを読み込みます。

In [ ]:
dataset_cv = load_dataset("food101", split="train[:100]")
dataset_cv[0]["image"]

In [ ]:
# model: google/vit-base-patch16-224 (86.6M params)
# ref: https://huggingface.co/google/vit-base-patch16-224

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

続いて、データセットに含まれる画像データを、所定の方法で処理していきます。
ここでは、`torchvision` の `transforms` モジュールを使用します。
ここで行う処理は以下のとおりです。

- `RandomResizedCrop`: モデルが期待する画像サイズ (`image_size`) に合わせて、画像をランダムに切り抜く。
- `ColorJitter`: 画像の色調や明るさをランダムに変化させる。
    - `brightness=0.5`: 明るさを $\pm 50 \%$ の範囲でランダムに変化させる。
    - `hue=0.5`: 色相値を $\pm 0.5$ の範囲でランダムに変化させる。

In [ ]:
image_size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

transforms_core = Compose([
    RandomResizedCrop(image_size),
    ColorJitter(brightness=0.5, hue=0.5),
])

`transforms_core` をデータセットの各画像に適用し、画像プロセッサで処理します。

In [ ]:
def transforms(examples):
    images = [transforms_core(img.convert("RGB")) for img in examples["image"]]
    examples["pixel_values"] = image_processor(images, do_resize=False, return_tensors="pt")["pixel_values"]
    return examples

In [ ]:
dataset_cv.set_transform(transforms)
dataset_cv[0].keys()

In [ ]:
img = dataset_cv[0]["pixel_values"]
plt.imshow(img.permute(1, 2, 0))

### Padding

データセットに含まれる画像のサイズが異なる場合には、`DataImageProcessor.pad()` によってパディングを施します。

In [ ]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = labels
    return batch

## Multi Modal

マルチモーダルタスクにおいて、データの事前処理に使用する主なアーキテクチャは**プロセッサ**です。 プロセッサはトーカナイザや特徴抽出器などの複数の事前処理アーキテクチャを結合します。

まずはデータセットを読み込みます。
[オリジナルのチュートリアル](https://huggingface.co/docs/transformers/v4.57.1/ja/preprocessing) では `lj_speech` というデータセットが読み込まれていますが、このデータセットは最新の `datasets` (version 4.3.0) ではサポートされていないので、別のデータセットをダウンロードします。

In [ ]:
librispeech1 = load_dataset("hf-internal-testing/librispeech_asr_demo", split="validation")
librispeech1.features

今回興味があるのは `audio` と `text` だけなので、それ以外のメンバを削除してしまいます。

In [ ]:
librispeech2 = librispeech1.map(remove_columns=["file", "id", "chapter_id", "speaker_id"])
print(librispeech2.features)
print(librispeech2[0]["audio"]["sampling_rate"])
print(librispeech2[0]["text"])

`librispeech_asr_demo` データセットはサンプリングレートが 16 kHz でモデルの事前学習データセットのサンプリングレートと一致しているので、リサンプリングの必要はありません。
安心してモデルを読み込みましょう。

In [ ]:
# model: facebook/wav2vec2-base-960h (95M params)
# ref: https://huggingface.co/facebook/wav2vec2-base-960h

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

`processor` に `audio` と `text` を指定して、事前処理を行います。

In [ ]:
def prepare_dataset(example):
    audio = example["audio"]

    example.update(processor(audio=audio["array"], text=example["text"], sampling_rate=16000))

    return example

In [ ]:
prepare_dataset(librispeech2[0])